In [12]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import sys
sys.path.insert(1,'c:/Users/h2pro/Programming')
import StockPull
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import seaborn as sns
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

import torch
import torch.nn as nn

# %%
#load data
company = 'MSFT'
start = dt.datetime(2012,1,1)
end = dt.datetime(2022,1,1)

data = StockPull.analysisout(company)
df = data

# %%
#Separate dates for future plotting
train_dates = pd.to_datetime(df['Date'])
print(train_dates.tail(15)) #Check last few dates. 

#Variables for training
cols = list(df)[1:3]
#New dataframe with only training data - 5 columns
df_for_training = df[cols].astype(float)
# print(train_data)
print(df_for_training)

# %%
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

# %%
#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 60  # Number of past days we want to use to predict the future.



1244   2022-12-30 00:00:00-05:00
1245   2023-01-03 00:00:00-05:00
1246   2023-01-04 00:00:00-05:00
1247   2023-01-05 00:00:00-05:00
1248   2023-01-06 00:00:00-05:00
1249   2023-01-09 00:00:00-05:00
1250   2023-01-10 00:00:00-05:00
1251   2023-01-11 00:00:00-05:00
1252   2023-01-12 00:00:00-05:00
1253   2023-01-13 00:00:00-05:00
1254   2023-01-17 00:00:00-05:00
1255   2023-01-18 00:00:00-05:00
1256   2023-01-19 00:00:00-05:00
1257   2023-01-20 00:00:00-05:00
1258   2023-01-23 00:00:00-05:00
Name: Date, dtype: datetime64[ns, America/New_York]
           Price      Volume
0      86.543587  23412800.0
1      86.468246  33277500.0
2      86.948517  26383200.0
3      88.577698  29172200.0
4      88.445854  31569900.0
...          ...         ...
1254  240.350006  29831300.0
1255  235.809998  30028700.0
1256  231.929993  28623000.0
1257  240.220001  35323400.0
1258  244.270004  15601977.0

[1259 rows x 2 columns]


In [26]:
#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

# %%
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, return_sequences=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, num_layers=2, batch_first=True, return_sequences=False)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [23]:
input_size = trainX.shape[2]
hidden_size = 64
output_size = 1

model = LSTMModel(input_size, hidden_size, output_size)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 25
batch_size = 16
validation_split = 0.1

trainX = torch.tensor(trainX, dtype=torch.float32)
trainY = torch.tensor(trainY, dtype=torch.float32)

train_dataset = torch.utils.data.TensorDataset(trainX, trainY)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = torch.utils.data.TensorDataset(trainX, trainY)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0

TypeError: RNNBase.__init__() got an unexpected keyword argument 'return_sequences'

In [15]:
# Training
for i, (inputs, labels) in enumerate(train_loader):
    optimizer.zero_grad()

    outputs = model(inputs)
    loss = loss_fn(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

# Validation
for i, (inputs, labels) in enumerate(valid_loader):
    outputs = model(inputs)
    loss = loss_fn(outputs, labels)

    valid_loss += loss.item()

print('Epoch {}/{}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch+1, num_epochs, train_loss/len(train_loader), valid_loss/len(valid_loader)))

NameError: name 'train_loader' is not defined